<a href="https://colab.research.google.com/github/veegaaa/nlp_course/blob/master/week1_embeddings/seminar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Seminar 1: Fun with Word Embeddings (3 points)

Today we gonna play with word embeddings: train our own little embedding, load one from   gensim model zoo and use it to visualize text corpora.

This whole thing is gonna happen on top of embedding dataset.

__Requirements:__  `pip install --upgrade nltk gensim bokeh` , but only if you're running locally.

In [0]:
!pip install --upgrade bokeh

    100% |████████████████████████████████| 16.3MB 2.0MB/s 
  Running setup.py bdist_wheel for bokeh ... - \ | / - \ | / - \ | / done
  Stored in directory: /root/.cache/pip/wheels/a2/0d/1e/d5ecb93d15d5c7558fb3ac84018487fc1b24de63b2459a7822
Successfully built bokeh
  Found existing installation: bokeh 1.0.2
    Uninstalling bokeh-1.0.2:
      Successfully uninstalled bokeh-1.0.2


In [0]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

--2019-01-02 18:35:55--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/obaitrix9jyu84r/quora.txt [following]
--2019-01-02 18:35:56--  https://www.dropbox.com/s/dl/obaitrix9jyu84r/quora.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc94471a4c13840114618b297f71.dl.dropboxusercontent.com/cd/0/get/AYq1I2XsMwT_aV74rESnK1TeqCm1hh-c_U0hoXsaVpZPta8YX9CiTOfqVeWKfdTbSHG7CpXx-yNT1Y0mUsfOKlubZ1ijQ2taZBB4htWTzheV2yJ3-fSOyqlFInhkD2kydDauzmUrzBq4Ms8V4FN0XFrsvds_gc8zgrcETAbTumFM7JxeU-WM7bzVEkxVeIiW5Eg/file?dl=1 [following]
--2019-01-02 18:35:56--  https://uc94471a4c13840114618b297f71.dl.dropboxusercontent.com/cd/0/get/AYq1I2XsMwT_aV74rESnK1TeqCm1hh-c_U0hoXsaVpZPta8YX9CiTOfqVe

In [0]:
import numpy as np

data = list(open("./quora.txt"))
data[50]

"What TV shows or books help you read people's body language?\n"

In [0]:
len(data)

537272

__Tokenization:__ a typical first step for an nlp task is to split raw data into words.
The text we're working with is in raw format: with all the punctuation and smiles attached to some words, so a simple str.split won't do.

Let's use __`nltk`__ - a library that handles many nlp tasks like tokenization, stemming or part-of-speech tagging.

In [0]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [0]:
# TASK: lowercase everything and extract tokens with tokenizer. 
# data_tok should be a list of lists of tokens for each line in data.

data_tok = [tokenizer.tokenize(line.lower()) for line in data]

In [0]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [0]:
print([' '.join(row) for row in data_tok[:2]])

["can i get back with my ex even though she is pregnant with another guy ' s baby ?", 'what are some ways to overcome a fast food addiction ?']


__Word vectors:__ as the saying goes, there's more than one way to train word embeddings. There's Word2Vec and GloVe with different objective functions. Then there's fasttext that uses character-level models to train word embeddings. 

The choice is huge, so let's start someplace small: __gensim__ is another nlp library that features many vector-based models incuding word2vec.

In [0]:
%%time
from gensim.models import Word2Vec
model = Word2Vec(data_tok, 
                 size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

CPU times: user 1min 10s, sys: 440 ms, total: 1min 10s
Wall time: 38.5 s


In [0]:
# now you can get word vectors !
model.get_vector('anything')

array([ 0.9485703 , -4.6517816 ,  0.42198455,  0.8093643 , -3.3987057 ,
       -0.2415365 , -2.0676417 , -2.2694297 ,  1.7848715 ,  0.01976249,
        1.9844956 ,  3.5309265 , -1.8488277 ,  2.9315069 , -0.9532372 ,
       -0.11393447, -1.894777  , -0.2757984 , -0.14268841, -1.8390828 ,
       -0.18169177, -2.8958938 , -3.040911  , -1.8668197 ,  1.0904202 ,
       -0.4530354 , -1.8500684 ,  2.739538  ,  0.6331613 , -2.8107035 ,
       -1.159934  ,  0.13013108], dtype=float32)

In [0]:
# or query similar words directly. Go play with it!
model.most_similar('bread')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('rice', 0.9525082111358643),
 ('beans', 0.9313331842422485),
 ('noodles', 0.9199264645576477),
 ('fruit', 0.9195559024810791),
 ('cheese', 0.9194595217704773),
 ('corn', 0.9160299301147461),
 ('garlic', 0.9116026163101196),
 ('butter', 0.9096128344535828),
 ('chocolate', 0.9075407385826111),
 ('pasta', 0.9046938419342041)]

In [0]:
# from gensim.models import Word2Vec
# w2v = Word2Vec(data_tok, 
#                  size=32,    
#                  min_count=5,
#                  window=5)

### Using pre-trained model

Took it a while, huh? Now imagine training life-sized (100~300D) word embeddings on gigabytes of text: wikipedia articles or twitter posts. 

Thankfully, nowadays you can get a pre-trained word embedding model in 2 lines of code (no sms required, promise).

In [0]:
import gensim.downloader as api
model1 = api.load('glove-twitter-25')

[==================================================] 100.0% 104.8/104.8MB downloaded


In [0]:
model1.most_similar(positive=["coder", "money"], negative=["brain"])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('realtor', 0.8265186548233032),
 ('gfx', 0.8249695897102356),
 ('caterers', 0.798202395439148),
 ('beatmaker', 0.7936854362487793),
 ('recruiter', 0.7892400622367859),
 ('sfi', 0.784467339515686),
 ('sosh', 0.7840631604194641),
 ('promoter', 0.7838252186775208),
 ('smallbusiness', 0.7786215543746948),
 ('promoters', 0.7764680981636047)]

In [0]:
model = model1

In [0]:
import gensim.downloader as api
model = api.load('glove-twitter-100')

In [0]:
model.most_similar(positive=["coder", "money"], negative=["brain"])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('broker', 0.5820155739784241),
 ('bonuses', 0.5424473285675049),
 ('banker', 0.538511335849762),
 ('designer', 0.5197198390960693),
 ('merchandising', 0.4964233934879303),
 ('treet', 0.49220192432403564),
 ('shopper', 0.4920561909675598),
 ('part-time', 0.49128279089927673),
 ('freelance', 0.4843311905860901),
 ('aupair', 0.4796452522277832)]

### Visualizing word vectors

One way to see if our vectors are any good is to plot them. Thing is, those vectors are in 30D+ space and we humans are more used to 2-3D.

Luckily, we machine learners know about __dimensionality reduction__ methods.

Let's use that to plot 1000 most frequent words

In [0]:
words = sorted(model.vocab.keys(), 
               key=lambda word: model.vocab[word].count,
               reverse=True)[:1000]

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [0]:
# for each word, compute it's vector with model
word_vectors = np.array([model.get_vector(word) for word in words])

In [0]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### Linear projection: PCA

The simplest linear dimensionality reduction method is __P__rincipial __C__omponent __A__nalysis.

In geometric terms, PCA tries to find axes along which most of the variance occurs. The "natural" axes, if you wish.

<img src="https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/pca_fish.png" style="width:30%">


Under the hood, it attempts to decompose object-feature matrix $X$ into two smaller matrices: $W$ and $\hat W$ minimizing _mean squared error_:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ - object matrix (**centered**);
- $W \in \mathbb{R}^{m \times d}$ - matrix of direct transformation;
- $\hat{W} \in \mathbb{R}^{d \times m}$ - matrix of reverse transformation;
- $n$ samples, $m$ original dimensions and $d$ target dimensions;



In [0]:
word_vectors.shape

(1000, 100)

In [0]:
from sklearn.decomposition import PCA

# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
# after that, normalize vectors to make sure they have zero mean and unit variance
pca = PCA(2)

# and maybe MORE OF YOUR CODE here :)
word_vectors_pca = pca.fit_transform(word_vectors)
word_vectors_pca = (word_vectors_pca - word_vectors_pca.mean(axis=0).reshape(-1,2)) / word_vectors_pca.std(axis=0).reshape(-1,2)

In [0]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

#### Let's draw it!

In [0]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    output_notebook()
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [0]:
# output_notebook()
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

Loading BokehJS ...

Figure(id='1235', ...)

### Visualizing neighbors with t-SNE
PCA is nice but it's strictly linear and thus only able to capture coarse high-level structure of the data.

If we instead want to focus on keeping neighboring points near, we could use TSNE, which is itself an embedding method. Here you can read __[more on TSNE](https://distill.pub/2016/misread-tsne/)__.

In [0]:
from sklearn.manifold import TSNE

# map word vectors onto 2d plane with TSNE. hint: use verbose=100 to see what it's doing.
# normalize them as just lke with pca


word_tsne = TSNE().fit_transform(word_vectors)

In [0]:
output_notebook()
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

Loading BokehJS ...

Loading BokehJS ...

Figure(id='1115', ...)

### Visualizing phrases

Word embeddings can also be used to represent short phrases. The simplest way is to take __an average__ of vectors for all tokens in the phrase with some weights.

This trick is useful to identify what data are you working with: find if there are any outliers, clusters or other artefacts.

Let's try this new hammer on our data!


In [0]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros
    
    vector = np.zeros([model.vector_size], dtype='float32')
    
    # YOUR CODE
    wv_list = []
    for word in tokenizer.tokenize(phrase.lower()):
      if word in model.vocab:
        wv_list.append(model.get_vector(word))
    if len(wv_list) == 0:
        wv_list.append(np.zeros(model.vector_size))
    vector = np.mean(wv_list, axis=0)
    return vector
        
    

In [0]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros
    
    vector = np.zeros([model.vector_size], dtype='float32')
    
    # YOUR CODE
    wv_list = []
    for word in tokenizer.tokenize(phrase.lower()):
      if word in model.vocab:
        vector += model.get_vector(word)
    return vector
        
    

In [0]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")

assert np.allclose(vector[::10],
                   np.array([ 0.31807372, -0.02558171,  0.0933293 , -0.1002182 , -1.0278689 ,
                             -0.16621883,  0.05083408,  0.17989802,  1.3701859 ,  0.08655966],
                              dtype=np.float32))

In [0]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::len(data) // 1000]

# compute vectors for chosen phrases
phrase_vectors = np.array([get_phrase_embedding(phrase) for phrase in chosen_phrases])

In [0]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [0]:
# # map vectors into 2d space with pca, tsne or your other method of choice
# # don't forget to normalize

# phrase_vectors_2d = TSNE(verbose=1000).fit_transform(phrase_vectors)

# phrase_vectors_2d = (phrase_vectors_2d - phrase_vectors_2d.mean(axis=0)) / phrase_vectors_2d.std(axis=0)

In [0]:
# draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
#              phrase=[phrase[:50] for phrase in chosen_phrases],
#              radius=20,)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [0]:
# compute vector embedding for all lines in data`
data_vectors = np.array([get_phrase_embedding(l) for l in data])

In [0]:
from sklearn import metrics

In [0]:
data_vectors.shape

(537272, 100)

In [0]:
# def find_nearest(query, k=10):
#     """
#     given text line (query), return k most similar lines from data, sorted from most to least similar
#     similarity should be measured as cosine between query and line embedding vectors
#     hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition(), np.argsort
#     """
#     # YOUR CODE
#     query_embed = get_phrase_embedding(query)[None,:]
#     distances = metrics.pairwise.cosine_distances(data_vectors, query_embed)

#     return [data[i] for i in np.argsort(distances[:, 0])[:k]]


In [0]:
def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition(), np.argsort
    """
    # YOUR CODE
    query_embed = get_phrase_embedding(query)
    distances = (data_vectors*query_embed).sum(axis=1) / \
    np.sqrt((query_embed*query_embed).sum()) / \
    np.sqrt((data_vectors*data_vectors).sum(axis=1))
    
    
    return [data[i] for i in (1-distances).argsort()[:k]]

In [0]:
find_nearest(query="Trump", k=10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


['Will Donald Trump destroy Donald Trump?\n',
 'Why is Quora biased agaist Donald Trump.?\n',
 'Donald Trump or Hillary Clinton?\n',
 'Donald Trump: Does Donald Trump have misogynistic views?\n',
 'How did Melania Trump meet Donald Trump?\n',
 'Donald trump father?\n',
 'Donald Trump India?\n',
 'Is Donald Trump xenophobic?\n',
 'Does Donald Trump use Quora?\n',
 'Is Donald Trump authoritarian?\n']

In [0]:
results = find_nearest(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'
assert results[3] == 'What can I do to save the world?\n'

How do I get to the dark web?
What should I do to enter hollywood?
How do I use the Greenify app?
What can I do to save the world?
How do I win this?
How do I think out of the box? How do I learn to think out of the box?
How do I find the 5th dimension?
How do I use the pad in MMA?
How do I estimate the competition?
What do I do to enter the line of event management?



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


In [0]:
results = find_nearest(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'
assert results[3] == 'What can I do to save the world?\n'

How do I get to the dark web?
What should I do to enter hollywood?
How do I use the Greenify app?
What can I do to save the world?
How do I win this?
How do I think out of the box? How do I learn to think out of the box?
How do I find the 5th dimension?
How do I use the pad in MMA?
How do I estimate the competition?
What do I do to enter the line of event management?



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


In [0]:
find_nearest(query="How does Trump?", k=10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


['What does Donald Trump think about Israel?\n',
 'What books does Donald Trump like?\n',
 'What does Donald Trump think of India?\n',
 'What does India think of Donald Trump?\n',
 'What does Donald Trump think of China?\n',
 'What does Donald Trump think about Pakistan?\n',
 'What companies does Donald Trump own?\n',
 'What does Dushka Zapata think about Donald Trump?\n',
 'How does it feel to date Ivanka Trump?\n',
 'What does salesforce mean?\n']

In [0]:
find_nearest(query="Why don't i ask a question myself?", k=10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


["Why don't I get a date?\n",
 "Why do you always answer a question with a question? I don't, or do I?\n",
 "Why can't I ask a question anonymously?\n",
 "Why don't I get a girlfriend?\n",
 "Why don't I have a boyfriend?\n",
 "I don't have no question?\n",
 "Why can't I take a joke?\n",
 "Why don't I ever get a girl?\n",
 "Can I ask a girl out that I don't know?\n",
 "Why don't I have a girlfriend?\n"]

__Now what?__
* Try running TSNE on all data, not just 1000 phrases
* See what other embeddings are there in the model zoo: `gensim.downloader.info()`
* Take a look at [FastText](https://github.com/facebookresearch/fastText) embeddings
* Optimize find_nearest with locality-sensitive hashing: use [nearpy](https://github.com/pixelogik/NearPy) or `sklearn.neighbors`.